In [1]:
from sqlalchemy import create_engine, Table, MetaData, insert, text, inspect
from sqlalchemy.orm import sessionmaker
import pandas as pd
import json

In [2]:
DATABASE_URL = f"mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)
metadata = MetaData()
paximum_table = Table('juniper', metadata, autoload_with=engine)

In [3]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT DATABASE()"))
        db_name = result.fetchone()[0]
        print(f"Connected to database: {db_name}")
except Exception as e:
    print(f"Error: {e}")


Connected to database: csvdata01_02102024


In [4]:
try:
    inspect_table = inspect(engine)
    tables = inspect_table.get_table_names()
    if tables:
        for table in tables:
            print(table)

except Exception as e:
    print(f"Error {e}")

hotels_info
hotels_info_with_gidestination_code
info_01
innova_hotels_main
juniper
my_table
paximum
ratehawk
z_ocas_member_data


In [5]:
import requests
import xmltodict
import json

url = "https://xml-uat.bookingengine.es/WebService/jp/operations/staticdatatransactions.asmx"

payload = "<soapenv:Envelope xmlns:soapenv=\"http://schemas.xmlsoap.org/soap/envelope/\" xmlns=\"http://www.juniper.es/webservice/2007/\">\n   <soapenv:Header/>\n   <soapenv:Body>\n      <HotelPortfolio>\n         <HotelPortfolioRQ Version=\"1.1\" Language=\"en\" Page=\"2\" RecordsPerPage=\"5\">\n            <Login Password=\"^wvW6#[49J/G\" Email=\"TestXMLNofshonTours\"/>\n         </HotelPortfolioRQ>\n      </HotelPortfolio>\n   </soapenv:Body>\n</soapenv:Envelope>"
headers = {
  'Content-Type': 'text/xml;charset=UTF-8',
  'SOAPAction': '"http://www.juniper.es/webservice/2007/HotelPortfolio"',
  'Cookie': 'StatC=8l+HAPXk1NeIsJR8qq/ISw==; StatP=lh5UxVPejVDjU3b27uc0fygTysSaay6K5gfbVxJWTYZM4f+TafxAFFOXsj/1uQ3n; idioma=en'
}

response = requests.request("POST", url, headers=headers, data=payload)

data_dict = xmltodict.parse(response.text)
print(data_dict)


{'soap:Envelope': {'@xmlns:soap': 'http://schemas.xmlsoap.org/soap/envelope/', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema', 'soap:Body': {'HotelPortfolioResponse': {'@xmlns': 'http://www.juniper.es/webservice/2007/', 'HotelPortfolioRS': {'@Url': 'http://xml-uat.bookingengine.es', '@TimeStamp': '2024-11-24T09:45:20.8084722+01:00', '@IntCode': 'eB1PK6kkSGDXlhHU5/ualc7XByufFDWFmVkBx1qbMSQ=', 'HotelPortfolio': {'@Page': '2', '@TotalPages': '1952', '@TotalRecords': '9757', '@LastUpdated': '2024-09-05T20:00:59.153', 'Hotel': [{'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060468', '@HasSynonyms': 'false', 'Name': 'Embat', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer Nanses, 7, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5343124', 'Longitude': '2.7144459', 'HotelCategory': {'@Type': '2Est', '@Code': '2', '#text'

In [13]:

hotel_content = data_dict['soap:Envelope']['soap:Body']['HotelPortfolioResponse']["HotelPortfolioRS"]['HotelPortfolio']['Hotel']

print(hotel_content)

[{'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060468', '@HasSynonyms': 'false', 'Name': 'Embat', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer Nanses, 7, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5343124', 'Longitude': '2.7144459', 'HotelCategory': {'@Type': '2Est', '@Code': '2', '#text': '2 Stars'}, 'City': {'@Id': '15011', '@JPDCode': 'JPD054557', '#text': 'Palma de Mallorca'}}, {'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060489', '@HasSynonyms': 'false', 'Name': 'Isla del Sol Apartments', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer dels Pins, 5, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5375776', 'Longitude': '2.7142996', 'HotelCategory': {'@Code': '8'}, 'City': {'@Id': '15011', '@JPDCode': 'JPD054557', '#text': 'Palma de Mallorca'}}, {'@

In [14]:
# Extract hotel data
hotels = []
hotel_list = data_dict.get("soap:Envelope", {}).get("soap:Body", {}).get(
    "HotelPortfolioResponse", {}
).get("HotelPortfolioRS", {}).get("HotelPortfolio", {}).get("Hotel", [])

# print(hotel_list)

if not hotel_list:
    print("No hotels found in the response.")
    exit()

# Process and insert hotel data into the database
with engine.connect() as conn:
    for hotel in hotel_list:
        hotel_data = {
            "HotelName": hotel.get("Name"),
            "HotelId": hotel.get("@JPCode"),
            "ZipCode": hotel.get("ZipCode"),
            "Latitude": hotel.get("Latitude"),
            "Longitude": hotel.get("Longitude"),
        }

        # Insert into the database
        try:
            stmt = insert(juniper_table).values(hotel_data)
            conn.execute(stmt)
            conn.commit()  # Save the changes immediately
            print(f"Inserted: {hotel_data}")
        except Exception as e:
            print(f"Error inserting {hotel_data}: {e}")

[{'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060468', '@HasSynonyms': 'false', 'Name': 'Embat', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer Nanses, 7, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5343124', 'Longitude': '2.7144459', 'HotelCategory': {'@Type': '2Est', '@Code': '2', '#text': '2 Stars'}, 'City': {'@Id': '15011', '@JPDCode': 'JPD054557', '#text': 'Palma de Mallorca'}}, {'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060489', '@HasSynonyms': 'false', 'Name': 'Isla del Sol Apartments', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer dels Pins, 5, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5375776', 'Longitude': '2.7142996', 'HotelCategory': {'@Code': '8'}, 'City': {'@Id': '15011', '@JPDCode': 'JPD054557', '#text': 'Palma de Mallorca'}}, {'@

In [14]:
import requests
import xmltodict
import json

url = "https://xml-uat.bookingengine.es/WebService/jp/operations/staticdatatransactions.asmx"

payload = "<soapenv:Envelope xmlns:soapenv=\"http://schemas.xmlsoap.org/soap/envelope/\" xmlns=\"http://www.juniper.es/webservice/2007/\">\n    <soapenv:Header/>\n    <soapenv:Body>\n        <HotelContent>\n            <HotelContentRQ Version=\"1\" Language=\"en\">\n                <Login Password=\"^wvW6#[49J/G\" Email=\"TestXMLNofshonTours\"/>\n                <HotelContentList>\n                    <Hotel Code=\"JP146952\"/>\n                </HotelContentList>\n            </HotelContentRQ>\n        </HotelContent>\n    </soapenv:Body>\n</soapenv:Envelope>"
headers = {
  'Content-Type': 'text/xml;charset=UTF-8',
  'SOAPAction': '"http://www.juniper.es/webservice/2007/HotelContent"',
  'Cookie': 'StatC=8l+HAPXk1NeIsJR8qq/ISw==; StatP=UAvxTZg5/KSb5xjepzejGq+uRWuHJ5ucWURdLMNyct+5jbHBZLi9nj/RVgL8/LVZ; idioma=en'
}

response = requests.request("POST", url, headers=headers, data=payload)

# print(response.text)
data_dict = xmltodict.parse(response.text)
print(data_dict)


{'soap:Envelope': {'@xmlns:soap': 'http://schemas.xmlsoap.org/soap/envelope/', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema', 'soap:Body': {'HotelContentResponse': {'@xmlns': 'http://www.juniper.es/webservice/2007/', 'ContentRS': {'@Url': 'http://xml-uat.bookingengine.es', '@TimeStamp': '2024-11-24T10:00:54.663919+01:00', '@IntCode': 'W1jPfHLa7yw63sLhTRyKf6tcTYzXE9kT4LD5fTfBNgA=', 'Contents': {'HotelContent': {'@Code': 'JP146952', '@JPCode': 'JP146952', 'HotelName': 'Apart Hotel Serantes', 'Zone': {'@JPDCode': 'JPD173601', '@Code': '131874', 'Name': 'Natal, Rio Grande do Norte, Brazil'}, 'HotelChain': {'Name': '0|No chain|'}, 'HotelCategory': {'@Type': '3est', '#text': '3 Stars'}, 'Address': {'Address': 'R. Dr. Silvino Meira e Sá Bezerra, 130 - Vila de Ponta negra, Natal - RN, 59090-170, Brazil', 'PostalCode': '59090-170', 'Latitude': '-5.8814708', 'Longitude': '-35.1733337'}, 'Images': {'Image': [{'@Type': 'BIG', 'FileName'

In [19]:
data_dict = xmltodict.parse(response.text)

content = data_dict['soap:Envelope']['soap:Body']['HotelContentResponse']['ContentRS']['Contents']['HotelContent']
data = content.get('Features', {})
print(data)

{'Feature': [{'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'English'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Portuguese'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Spanish'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Wheelchair accessible'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Free Wi-Fi in all rooms!'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Internet'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Internet services'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Wi-Fi in public areas'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Swimming pool [outdoor]'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Tours'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Anti-viral cleaning products'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Body thermometer'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Breakfast takeaway service'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Cashless payment service'}, {'@Type'

## Local IIT table to server IIT Table.

In [3]:
from sqlalchemy import create_engine, Table, MetaData, insert, select
from sqlalchemy.exc import SQLAlchemyError
import os 
from dotenv import load_dotenv
import logging


logging.basicConfig(level=logging.ERROR, filename='error_log.log')

load_dotenv()

db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

LOCAL_DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
local_engine = create_engine(LOCAL_DATABASE_URL)

SERVER_DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
server_engine = create_engine(SERVER_DATABASE_URL)


def fetch_and_insert_data():
    local_connection = local_engine.connect()
    server_connection = server_engine.connect()

    metadata = MetaData()

    table_main_local = Table('innova_hotels_main', metadata, autoload_with=local_engine)

    query = select(table_main_local).where(table_main_local.c.SupplierCode == "Juniper Hotel")
    try:
        result = local_connection.execute(query).fetchall()
        print(f"Fetched {len(result)} records from the local database.")

        if result:
            server_table = Table('innova_hotels_main', metadata, autoload_with=server_engine)
            CHUNK_SIZE = 100
            total_inserted = 0

            for i in range(0, len(result), CHUNK_SIZE):
                chunk = result[i:i + CHUNK_SIZE]

                # Exclude the 'Id' field from each row
                prepared_chunk = [
                    {key: value for key, value in row._mapping.items() if key != 'Id'} 
                    for row in chunk
                ]
                
                with server_engine.begin() as transaction:
                    try:
                        insert_query = insert(server_table).values(prepared_chunk)
                        transaction.execute(insert_query)
                        total_inserted += len(prepared_chunk)
                        print(f"Inserted {len(prepared_chunk)} records into the server database. Total inserted so far: {i + len(prepared_chunk)}")
                    except SQLAlchemyError as e:
                        logging.error(f"An error occurred: {e}")
                        logging.error(f"Problematic chunk: {prepared_chunk}")
                        transaction.rollback()
                        break
        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        local_connection.close()
        server_connection.close()
        

fetch_and_insert_data()

Fetched 9359 records from the local database.
Inserted 100 records into the server database. Total inserted so far: 100
Inserted 100 records into the server database. Total inserted so far: 200
Inserted 100 records into the server database. Total inserted so far: 300
Inserted 100 records into the server database. Total inserted so far: 400
Inserted 100 records into the server database. Total inserted so far: 500
Inserted 100 records into the server database. Total inserted so far: 600
Inserted 100 records into the server database. Total inserted so far: 700
Inserted 100 records into the server database. Total inserted so far: 800
Inserted 100 records into the server database. Total inserted so far: 900
Inserted 100 records into the server database. Total inserted so far: 1000
Inserted 100 records into the server database. Total inserted so far: 1100
Inserted 100 records into the server database. Total inserted so far: 1200
Inserted 100 records into the server database. Total inserted s

### Get unique country name and country code.

In [12]:
from sqlalchemy import create_engine, MetaData, Table, text
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_pass = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

DATABASE_URL = f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}"
engine = create_engine(DATABASE_URL)

vervotech_mapping_table = 'vervotech_mapping'
country_code_table = 'country_code'

metadata = MetaData()
vervotech_mapping = Table(vervotech_mapping_table, metadata, autoload_with=engine)
country_code = Table(country_code_table, metadata, autoload_with=engine)

with engine.connect() as conn:
    query = text("""
        SELECT DISTINCT hotel_country, country_code
        FROM vervotech_mapping;
    """)
    result = conn.execute(query)
    

    country_data = [(row[0], row[1]) for row in result]
    print(len(country_data))

# with engine.connect() as conn:
#     with conn.begin(): 
#         for hotel_country_value, country_code_value in country_data:
#             insert_statement = country_code.insert().values(
#                 hotel_country=hotel_country_value,
#                 country_code=country_code_value
#             )
#             conn.execute(insert_statement)
#     conn.commit()

# print(f"Successfully inserted {len(country_data)} hotel_country and country_code values into {country_code_table} table.")


933
